<a href="https://colab.research.google.com/github/Edgar-La/hacking-civico/blob/master/proyecto/proyecto-hacking-civico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# __Titulo:__
En este documento encontras lo siguiente:

* Descripcion:

* Selección y obtención de los datos

* Exploración de los datos

* Visualización de los datos

* Conclusión 

* Referencias


## __Descripcion.__
Información de los registros de la deuda pública contraída vigente (acreedor, acreditado, monto, garantías, tasas, comisiones, plazos, amortizaciones, destino, etc.).
Auditoría Superior del Estado de Guanajuato

## __Seleccion y obtencion de datos.__

In [2]:
#Importar paquetes de Python
import pandas as pd
import numpy as np
import altair as alt
import statistics

In [ ]:
#Para descargar los datos
!wget http://www.aseg.gob.mx/descarga/deuda_publica.csv

In [229]:
#Convertir datos en DataFrame

# Especificamos el nombre del archivo por abrir
nombre_archivo = 'deuda_publica.csv'
# Utilizamos la función de Pandas para cargar un archivo CSV
Datos = pd.read_csv(nombre_archivo, encoding = 'latin-1')

In [243]:
Datos.dtypes

Ente_Acreditado                        object
No_de_Crédito                         float64
No_de_inscripción_SHCP                 object
Registro_Estatal                       object
Ambito                                 object
Institución_Acreedora                  object
Tipo_de_Obligación                     object
Clase_del_titulo                       object
Monto_original_contratado_en_pesos     object
Monto_dispuesto_en_pesos               object
Saldo_en_pesos                         object
Tasa                                   object
Fecha_de_Contratación                  object
Plazo_máximo_(meses)                   object
Fecha_de_Vencimiento                   object
Garantía                               object
Destino                                object
dtype: object

In [258]:
# Limpieza del nombre de las columnas, remover espacios y carácteres especiales como: '.' y '$'.
Datos.columns = Datos.columns.str.replace(' ', '_')
Datos.columns = Datos.columns.str.replace('.', '')
for t in Datos.Monto_original_contratado_en_pesos:
    Datos.Monto_original_contratado_en_pesos = Datos.Monto_original_contratado_en_pesos.str.replace('$', '')
for t in Datos.Monto_dispuesto_en_pesos:
    Datos.Monto_dispuesto_en_pesos = Datos.Monto_dispuesto_en_pesos.str.replace('$', '')
for t in Datos.Saldo_en_pesos	:
    Datos.Saldo_en_pesos	 = Datos.Saldo_en_pesos	.str.replace('$', '')


for t in Datos.Monto_original_contratado_en_pesos:
    Datos.Monto_original_contratado_en_pesos = Datos.Monto_original_contratado_en_pesos.str.replace(',', '')    
Datos   #Muestra cabecera

,Ente_Acreditado,No_de_Crédito,No_de_inscripción_SHCP,Registro_Estatal,Ambito,Institución_Acreedora,Tipo_de_Obligación,Clase_del_titulo,Monto_original_contratado_en_pesos,Monto_dispuesto_en_pesos,Saldo_en_pesos,Tasa,Fecha_de_Contratación,Plazo_máximo_(meses),Fecha_de_Vencimiento,Garantía,Destino
0,Gobierno del Estado de Guanajuato,NaN,167/2002,124/02,Estatal,"Banco del Bajío, S.A.",Contrato de Apertura de Crédito Simple,Pesos,233000000.00,"232,851,337.75","15,597,340.75",TIIE +1.00,09/12/2002,180,27/11/2017,Participaciones Federales,Se destinará precisa y exclusivamente para fin...
1,Gobierno del Estado de Guanajuato,1466.0,169/02,125/02,Estatal,Banobras S.N.C,Contrato de apertura de crédito simple y const...,Pesos,289037000.00,"289,037,000.00","52,301,933.78",FOAEM Nominal + 1.50,18/12/2002,240,10/03/2023,Participaciones Federales,Se destinará precisa y exclusivamente para fin...
2,Gobierno del Estado de Guanajuato,1467.0,169/02,125/02,Estatal,Banobras S.N.C,Contrato de apertura de crédito simple y const...,UDIS,289037000.00,"289,037,000.00","84,325,399.36",FOAEM Real + 1.50,18/12/2002,240,10/03/2023,Participaciones Federales,Se destinará precisa y exclusivamente para fin...
3,Tarimoro,4470.0,276/2004,140/04,Municipal,Banobras S.N.C,Contrato de apertura de crédito simple y const...,Pesos,3132842.97,"3,132,842.97","671,324.13",TIIE * 1.120,11/11/2004,180,23/12/2019,Participaciones Federales,Se destinará única y exclusivamente en los sig...
4,Gobierno del Estado de Guanajuato,5768.0,234/2005,147/05,Estatal,Banobras S.N.C,Contrato de Apertura de Crédito Simple,Pesos,1184414400.00,"1,184,414,400.00","97,109,245.00",CETES 182 + 1.34,20/05/2005,216,01/02/2022,Participaciones Federales,Obra pública
5,Gobierno del Estado de Guanajuato,6342.0,234/2005,147/05,Estatal,Banobras S.N.C,Contrato de Apertura de Crédito Simple,Pesos,1184414400.00,"1,184,414,400.00","88,311,746.67",CETES 182 + 1.33,20/05/2005,216,01/02/2022,Participaciones Federales,Financiar el programa de financiamiento a estr...
6,Gobierno del Estado de Guanajuato,6938.0,234/2005,147/05,Estatal,Banobras S.N.C,Contrato de Apertura de Crédito Simple,Pesos,1184414400.00,"1,184,414,400.00","36,064,550.65",CETES 182 + 1.08,20/05/2005,216,01/02/2022,Participaciones Federales,Financiar el programa de financiamiento a estr...
7,Gobierno del Estado de Guanajuato,7061.0,234/2006,147/05,Estatal,Banobras S.N.C,Contrato de Apertura de Crédito Simple,Pesos,1184414400.00,"1,184,414,400.00","106,248,939.08",CETES 182 + 1.11,20/05/2005,216,01/02/2022,Participaciones Federales,Financiar el programa de financiamiento a estr...
8,Gobierno del Estado de Guanajuato,7848.0,234/2006,147/05,Estatal,Banobras S.N.C,Contrato de Apertura de Crédito Simple,Pesos,1184414400.00,"1,184,414,400.00","84,958,370.54",CETES 182 + 1.22,20/05/2005,216,01/02/2022,Participaciones Federales,Financiar el programa de financiamiento a estr...
9,Jerécuaro,7073.0,352/2007,168/07,Municipal,Banobras S.N.C,Contrato de Apertura de Crédito Simple,Pesos,4382506.00,"4,382,506.00","191,225.21",TIIE + 1.81,07/12/2007,120,02/01/2018,Participaciones Federales,Financiar los siguientes conceptos: \nTramo 1....


In [257]:
pd.to_numeric(Datos.Monto_original_contratado_en_pesos)
Datos.dtypes

Ente_Acreditado                        object
No_de_Crédito                         float64
No_de_inscripción_SHCP                 object
Registro_Estatal                       object
Ambito                                 object
Institución_Acreedora                  object
Tipo_de_Obligación                     object
Clase_del_titulo                       object
Monto_original_contratado_en_pesos     object
Monto_dispuesto_en_pesos               object
Saldo_en_pesos                         object
Tasa                                   object
Fecha_de_Contratación                  object
Plazo_máximo_(meses)                   object
Fecha_de_Vencimiento                   object
Garantía                               object
Destino                                object
dtype: object

## __Exploracion de los datos.__

In [250]:
#Datos.groupby('Ente_Acreditado').sum()
Datos.groupby('Institución_Acreedora').Ente_Acreditado.count()      #Filtrado por institucion acredora
#Datos.query('Ente_Acreditado == "Guanajuato"').head()

Institución_Acreedora
BBVA Bancomer                                             6
Banamex                                                   7
Banco del Bajío, S.A.                                    19
Banobras S.N.C                                           19
Banorte                                                   1
Concesionaria del Acueducto El Zapotillo S.A. de C.V.     1
HSBC                                                      2
Scotiabank                                                1
Name: Ente_Acreditado, dtype: int64

In [249]:
#Filtro para Guanajuato
Datos.query('Ente_Acreditado == "Gobierno del Estado de Guanajuato"').Monto_dispuesto_en_pesos.value_counts()

1,184,414,400.00    5
500,000,000.00      3
289,037,000.00      2
185,950,160.00      1
310,689,997.93      1
1,664,477,330.00    1
644,000,000.00      1
232,851,337.75      1
487,161,920.75      1
1,363,280,290.12    1
200,000,000.00      1
1,380,000,000.00    1
336,719,709.88      1
Name: Monto_dispuesto_en_pesos, dtype: int64

In [251]:
Datos_L = Datos.groupby('Ente_Acreditado').Monto_dispuesto_en_pesos.describe();
#Datos_L = Datos_L.convert_dtypes;
Datos_L

,count,unique,top,freq
Ente_Acreditado,,,,
Abasolo,1,1,"13,000,000.00",1
Apaseo el Grande,1,1,"12,559,400.30",1
Celaya,0,0,NaN,NaN
"Comisión Estatal del Agua de Guanajuato\nSistema de Agua Potable y Alcantarillado de León, Gto.",0,0,NaN,NaN
"Comité Municipal de Agua Potable y Alcantarillado de Salamanca, Gto.",1,1,"37,051,609.32",1
Comonfort,1,1,"19,860,880.00",1
Cuerámaro,2,2,"8,200,000.00",1
Doctor Mora,1,1,"7,493,848.97",1
Gobierno del Estado de Guanajuato,20,13,"1,184,414,400.00",5


In [252]:
Datos_M = Datos.groupby('Ente_Acreditado').Monto_dispuesto_en_pesos.value_counts()
#Datos_M = pd.DataFrame(data = Datos_M);
Datos_M

Ente_Acreditado                                                       Monto_dispuesto_en_pesos
Abasolo                                                               13,000,000.00               1
Apaseo el Grande                                                      12,559,400.30               1
Comité Municipal de Agua Potable y Alcantarillado de Salamanca, Gto.  37,051,609.32               1
Comonfort                                                             19,860,880.00               1
Cuerámaro                                                             7,300,000.00                1
                                                                      8,200,000.00                1
Doctor Mora                                                           7,493,848.97                1
Gobierno del Estado de Guanajuato                                     1,184,414,400.00            5
                                                                      500,000,000.00              3
     

In [216]:
#Datos_K = Datos.query('Ente_Acreditado == "Guanajuato"').Monto_dispuesto_en_pesos.value_counts()
#Datos_K

## __Visualizacion de los datos.__

In [253]:
Grafico_1 = alt.Chart(Datos).mark_bar().encode(
    x = alt.X('Fecha_de_Vencimiento:N'),
    y = alt.Y('Ente_Acreditado:N'),
    color = 'Ente_Acreditado',
)
Grafico_1

alt.Chart(...)

In [254]:
Grafico_2 = alt.Chart(Datos).mark_bar().encode(
    x = 'Ente_Acreditado',
    y = 'Monto_dispuesto_en_pesos',
    color = 'Ente_Acreditado'
)
Grafico_2

alt.Chart(...)

In [255]:
alt.Chart(Datos).mark_bar().encode(
    x = alt.X('Institución_Acreedora'),
    y = alt.X('count(Institución_Acreedora)'),
    color = 'Institución_Acreedora',
)

alt.Chart(...)

## __Conclusion.__

## __Referencias.__